In [1]:
%matplotlib inline
#! /usr/bin/env python
#"""Run a YOLO_v2 style detection model on test images."""
import argparse
import colorsys
import imghdr
import os
import random

import matplotlib.pyplot as plt
from scipy.misc import imresize
import numpy as np
from keras import backend as K
from keras.models import load_model
from PIL import Image, ImageDraw, ImageFont
import cv2
import matplotlib.image as mpimg
import glob
import pickle

from yad2k.models.keras_yolo import yolo_eval, yolo_head
from lane_finding import *

Using TensorFlow backend.


In [2]:
#Cofigure YOLOV2
model_path = 'model_data\\yolo-voc.h5'
assert model_path.endswith('.h5'), 'Keras model must be a .h5 file.'
anchors_path = 'model_data\\yolo-voc_anchors.txt'
classes_path = 'model_data\\pascal_classes.txt'
test_path = 'images\\'
output_path = 'images_out\\'

if not os.path.exists(output_path):
    print('Creating output path {}'.format(output_path))
    os.mkdir(output_path)

sess = K.get_session()  # TODO: Remove dependence on Tensorflow session.

with open(classes_path) as f:
    class_names = f.readlines()
class_names = [c.strip() for c in class_names]

with open(anchors_path) as f:
    anchors = f.readline()
    anchors = [float(x) for x in anchors.split(',')]
    anchors = np.array(anchors).reshape(-1, 2)

yolo_model = load_model(model_path)

# Verify model, anchors, and classes are compatible
num_classes = len(class_names)
num_anchors = len(anchors)
# TODO: Assumes dim ordering is channel last
model_output_channels = yolo_model.layers[-1].output_shape[-1]
assert model_output_channels == num_anchors * (num_classes + 5), \
    'Mismatch between model and given anchor and class sizes. ' \
    'Specify matching anchors and classes with --anchors_path and ' \
    '--classes_path flags.'
print('{} model, anchors, and classes loaded.'.format(model_path))

# Check if model is fully convolutional, assuming channel last order.
model_image_size = yolo_model.layers[0].input_shape[1:3]
is_fixed_size = model_image_size != (None, None)

# Generate colors for drawing bounding boxes.
hsv_tuples = [(x / len(class_names), 1., 1.)for x in range(len(class_names))]
colors = list(map(lambda x: colorsys.hsv_to_rgb(*x), hsv_tuples))
colors = list(map(lambda x: (int(x[0] * 255), int(x[1] * 255), int(x[2] * 255)),colors))
random.seed(10101)  # Fixed seed for consistent colors across runs.
random.shuffle(colors)  # Shuffle colors to decorrelate adjacent classes.
random.seed(None)  # Reset seed to default.

# Generate output tensor targets for filtered bounding boxes.
# TODO: Wrap these backend operations with Keras layers.
yolo_outputs = yolo_head(yolo_model.output, anchors, len(class_names))
input_image_shape = K.placeholder(shape=(2, ))
boxes, scores, classes = yolo_eval(yolo_outputs,input_image_shape,score_threshold=0.3,iou_threshold=0.5)

C:\ProgramData\Miniconda3\envs\carnd-term1-gpu\lib\site-packages\keras\models.py:257: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


model_data\yolo-voc.h5 model, anchors, and classes loaded.


In [3]:
def find_cars(image):
    # Load camera calibration
    mtx, dist = load_calibration('C:\carnd-term1\CarND-Advanced-Lane-Lines\camera_cal\wide_dist_pickle.p')
    # Undistort input image
    image = undistort(image, mtx, dist)
    box_image = np.zeros_like(image)
    image = Image.fromarray(image)
    box_image = Image.fromarray(box_image)
    
    if is_fixed_size:
        resized_image = image.resize(tuple(reversed(model_image_size)), Image.BICUBIC)
        image_data = np.array(resized_image, dtype='float32')
    else:
        # Due to skip connection + max pooling in YOLO_v2, inputs must have
        # width and height as multiples of 32.
        new_image_size = (image.width - (image.width % 32),image.height - (image.height % 32))
        resized_image = image.resize(new_image_size, Image.BICUBIC)
        image_data = np.array(resized_image, dtype='float32')

    image_data /= 255.
    image_data = np.expand_dims(image_data, 0)  # Add batch dimension.
    out_boxes, out_scores, out_classes = sess.run([boxes, scores, classes],
        feed_dict={
            yolo_model.input: image_data,
            input_image_shape: [image.size[1], image.size[0]],
            K.learning_phase(): 0
        })
    print('Found {} boxes'.format(len(out_boxes)))

    font = ImageFont.truetype(font='font/FiraMono-Medium.otf',size=np.floor(3e-2 * image.size[1] + 0.5).astype('int32'))
    thickness = (image.size[0] + image.size[1]) // 300

    for i, c in reversed(list(enumerate(out_classes))):
        predicted_class = class_names[c]
        box = out_boxes[i]
        score = out_scores[i]
        if predicted_class == 'car':
            label = '{} {:.2f}'.format(predicted_class, score)

            draw = ImageDraw.Draw(box_image)
            label_size = draw.textsize(label, font)

            top, left, bottom, right = box
            top = max(0, np.floor(top + 0.5).astype('int32'))
            left = max(0, np.floor(left + 0.5).astype('int32'))
            bottom = min(image.size[1], np.floor(bottom + 0.5).astype('int32'))
            right = min(image.size[0], np.floor(right + 0.5).astype('int32'))

            if top - label_size[1] >= 0:
                text_origin = np.array([left, top - label_size[1]])
            else:
                text_origin = np.array([left, top + 1])

            for i in range(thickness):
                draw.rectangle([left + i, top + i, right - i, bottom - i],outline=colors[c])
                draw.rectangle([tuple(text_origin), tuple(text_origin + label_size)],fill=colors[c])
                draw.text(text_origin, label, fill=(0, 0, 0), font=font)
            del draw
    image_out = np.asarray(box_image)
    return image_out

In [4]:
def process_image(image):
    lane_image = detect_lane(image)
    car_image = find_cars(image)
    image_out = cv2.addWeighted(lane_image, 1, car_image, 1, 0)
    
    return image_out

In [5]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

[WinError 5] Access is denied: 'C:\\ProgramData\\Miniconda3\\envs\\carnd-term1-gpu\\lib\\site-packages\\imageio\\resources\\ffmpeg\\ffmpeg.win32.exe'
  warn(skimage_deprecation('The `skimage.filter` module has been renamed '



In [6]:
left_line = Line()
right_line = Line()
video_output = 'project_video_out.mp4'
clip1 = VideoFileClip("project_video.mp4")
video_output_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time video_output_clip.write_videofile(video_output, audio=False)

Calculating Centroids


  if pixels_left == None:

  elif pixels_right == None:

  if pixels_left == None or pixels_right == None:

  lines_img[pixel[1],pixel[0]] = (0,255,0)

  lines_img[pixel[1],pixel[0]] = (0,255,0)



Found 0 boxes
[MoviePy] >>>> Building video project_video_out.mp4
[MoviePy] Writing video project_video_out.mp4


  0%|                                                                                         | 0/1261 [00:00<?, ?it/s]WARNING:py.warnings:C:\carnd-term1\CarND-Vehicle-Detection-P5\lane_finding.py:191: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  elif previous_fit != None:

  if pixels_left == None:

  elif pixels_right == None:

  if pixels_left == None or pixels_right == None:

  lines_img[pixel[1],pixel[0]] = (0,255,0)

  lines_img[pixel[1],pixel[0]] = (0,255,0)



Found 0 boxes


...

Sanity Check Failed due to curvature
12.6739739158
Found 2 boxes


100%|█████████████████████████████████████████████████████████████████████████████▉| 1259/1261 [08:06<00:00,  2.48it/s]

Sanity Check Failed due to curvature
-6.11056791136
Found 2 boxes


100%|█████████████████████████████████████████████████████████████████████████████▉| 1260/1261 [08:07<00:00,  2.47it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: project_video_out.mp4 

Wall time: 8min 8s
